In [1]:
from selenium import webdriver
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests

In [2]:
driver = webdriver.Chrome()
driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")
g_driver = webdriver.Chrome()

You can use this part to automatically login, but DUO two step authentication is still required. You will have to enter that manually. After entering the DUO code, you will be given a warning that the information you enter could be sent over an insecure connection. Click continue.

In [3]:
username = 'meb2fv'
password = '0ver In 2018!'
driver.find_element_by_id("user").send_keys(username);
driver.find_element_by_id("pass").send_keys(password);
driver.find_element_by_xpath("/html/body/main/div[2]/fieldset/form/input").click()

After you have logged in, Selenium is able to navigate to any webpage. We will navigate to the pages on the list:

In [4]:
def search_names(mid_first_name, last_name, school_url):
    link = 'https://heinonline-org.proxy01.its.virginia.edu/HOL/LuceneSearch?typea=title&termsa=&operator=AND&typeb=creator&termsb=' + last_name + '+' + mid_first_name + '&operatorb=AND&typec=text&termsc=&operatorc=AND&typed=title&termsd=&operatord=AND&typee=title&termse=&operatore=AND&typef=title&termsf=&yearlo=&yearhi=&tabfrom=&searchtype=field&collection=all&submit=Go'
    driver.get(link)
    try:
        webpage_wait('//*[@id="heinlogo"]/a/img')
        driver.find_element_by_xpath('//*[@id="search_modify"]/form/div/div/div/div/a[4]/i').click()
    except:
        driver.find_element_by_xpath('//*[@id="search_modify"]/div')
    element = driver.find_elements_by_tag_name('a')
    full_name = mid_first_name + ' ' +  last_name
    alt_fm_names = []
    err_fm_names = []
    if ' ' in mid_first_name.lower():
        first_name = mid_first_name.split(' ')[0]
    else: 
        first_name = mid_first_name
    page = 1
    
    while element:
        for link in element:
            link_text = link.text.lower()
            if first_name.lower() in link_text.lower() and last_name.lower() in link_text.lower() and '[' not in link_text:
                try:
                    new_last = link_text.split(', ')[0]
                    new_first_mid = link_text.split(', ')[1]
                    if first_name.lower() in new_first_mid and last_name.lower() == new_last:
                        new_fm = link.text.split(', ')[1]
                        if not new_fm in alt_fm_names and not new_fm in err_fm_names:
                            faculty = check_google(new_fm, last_name, school_url)
                            if faculty: 
                                alt_fm_names.append(new_fm)
                            else: 
                                err_fm_names.append(new_fm)
                except:
                    continue
        if page < 2:
            try:
                driver.find_element_by_xpath('//*[@id="thenext"]/span').click()
                time.sleep(3)
                element = driver.find_elements_by_tag_name('a') 
                page += 1
            except:
                element = []
        else: 
            element = []
    return alt_fm_names, err_fm_names

In [5]:
def similar_names(alt_name_list, err_fm_names, mid_first_name, last_name):
    try:
        driver.find_element_by_xpath('//*[@id="page_content"]/div[2]/div/b/a').click()
        element = driver.find_element_by_xpath('//*[@id="simlist"]/ul[1]')
        similar_name_list = [a.strip() for a in element.text.split('\n')]
        middle_name = ''
        if ' ' in mid_first_name.lower():
            first_name = mid_first_name.split(' ')[0]
            middle_name = mid_first_name.split(' ')[1]
        else: 
            first_name = mid_first_name
        for name in similar_name_list:
            if first_name.lower() in name.lower() and last_name.lower() in name.lower() and ', ' in name.lower():
                new_fm = name.split(', ', 1)[1]            
                new_last = name.split(', ', 1)[0]
                print(new_fm + ' ' + new_last)
                if new_fm not in alt_name_list and last_name.lower() == new_last.lower() and not new_fm in err_fm_names:
                    if ' ' in new_fm.lower() and middle_name != '':
                        new_mi = new_fm.split(' ')[1][0].lower()
                        if new_mi == middle_name[0].lower():
                            alt_name_list.append(new_fm)
                            continue
                    faculty = check_google(new_fm, last_name, school_url)
                    if faculty: 
                        alt_name_list.append(new_fm)
                    else: 
                        err_fm_names.append(new_fm)
    except:
        print('No similar names found.')
    return alt_name_list, err_fm_names

In [6]:
def check_google(mid_first_name, last_name, school_url):
    faculty = False
    for url in school_url:
        if faculty == True:
            break
        g_driver.get("http://google.com")
        search = g_driver.find_element_by_name('q')
        if not ' ' in mid_first_name:
            search.send_keys(mid_first_name + ' ' + last_name + ' ' + url)
        else: 
            search.send_keys(mid_first_name + ' ' + last_name + ' ')
        search.send_keys(Keys.RETURN)
        elems = g_driver.find_elements_by_xpath("//a[@href]")

        for elem in elems:
            if url in elem.text:
                print(url + ' in: ' + elem.text)
                faculty = True
                break
    return faculty

In [8]:
fm_name = 'Aaron'
last_name = 'Fellmeth'
alt_names, err_names = search_names(fm_name, last_name, ['asu'])
# alt_names, err_names = similar_names(alt_names, err_names, fm_name, last_name)
alt_names

asu in: Aaron Fellmeth | iSearch - ASU Faculty and Staff Search - Arizona ...
https://isearch.asu.edu/profile/730670
asu in: Aaron Fellmeth | iSearch
https://isearch.asu.edu/profile/730670
asu in: Aaron Fellmeth | iSearch
https://isearch.asu.edu/profile/730670


['Aaron', 'Aaron Xavier', 'Aaron X.']

If there is no middle name, a more famous person with the same first and last name will
    be at the top of the search results, so we need to add the school name
    In this case it will be ambiguous is another professor with the same first and last name 
    is in Hein anyway, so there is really no good way to tell if we are getting the wrong one.
    If they have a middle initial, I assume there is not another professor using the same exact name
    at a different school. Therefore, to avoid conflict with more famous people, I use the school name 
    in the search

In [8]:
def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

def to_float_or_int(input_list):
    new_list = []
    for x in input_list:
        x = x.replace(',','')
        try:
            value = int(x)
        except ValueError:
            try:
                value = float(x)
            except:
                value = ''
        new_list.append(value)
    return new_list

def create_dataframe_dict(my_list):#, person, school, data_type):
#     my_dict = {'Person': person, 
#               'School': school, 
#               'Type': data_type}
    my_dict = {}
    for stat in stats:
        my_dict[stat] = ''
        for item in my_list:   
            if item[0] == stat:
                my_dict[stat] = item[1]  
    return my_dict

def webpage_wait(xpath):
    element = []
    while not element:
        try:
            element = driver.find_element_by_xpath(xpath)
        except:
            print('Page has not loaded, sleeping for 3 seconds')
            time.sleep(3)

def remove_commas(df1):
    for col in df1.columns:
        df1[col] = df1[col].str.replace(',', '')
    return df1

def check_files(fm_name, last_name, current_files):
    done = False
    for cur_file in current_files:
        if fm_name.lower() in cur_file.lower() and last_name.lower() in cur_file.lower():
            done = True
            break
    return done

def check_df(current_stats, school_name):
    file = ''
    for cur_stat in current_stats:
        if school_name in cur_stat.lower():
            file = cur_stat
            break
    return file

def short_url(data):
    url_list = []
    for url in data:
        if 'https://www.' in url:
            new_url = url.split('https://www.')[1].split('.edu')[0]
        elif 'http://www.' in url:
            new_url = url.split('http://www.')[1].split('.edu')[0]
        elif 'www.' in url:
            new_url = url.split('www.')[1].split('.edu')[0]
        elif 'https://' in url:
            new_url = url.split('https://')[1].split('.edu')[0]
        elif 'http://' in url:
            new_url = url.split('http://')[1].split('.edu')[0]
        url_list.append(new_url)
    return url_list

In [9]:
data = pd.read_excel(create_path('Professor Names.xlsx'))
urls = pd.read_csv(create_path('University and College Websites update.csv'))
urls.head()

name_tuple = list(zip(data.First, data.Middle, data.Last))
fm_name = [x[0] + ' ' +  x[1]  if isinstance(x[1], str)  else x[0] for x in name_tuple ]
last_name = [x[2] for x in name_tuple ]
title = ['NA']*len(last_name)
new_data = pd.DataFrame({'First Name': fm_name, 'Last Name': last_name, 'Title': title, 'School': data.Former, 'New School': data.Current})

In [10]:
def get_school_urls(urls_df, school_list):
    url_list = []
    for school_name in school_list:
        school_name = school_name.replace(',', '')
        try: 
            index = urls_df[urls_df['School Name'] == school_name].index[0]
            print(school_name)
            url_name = urls_df['URL'][index]
            print(url_name)
            url_list.append(url_name)
        except:
            g_driver.get("http://google.com")
            search = g_driver.find_element_by_name('q')
            search.send_keys(school_name)
            search.send_keys(Keys.RETURN)
            element = g_driver.find_elements_by_xpath('//*[@id="rso"]/div[1]/div/div[1]/div/div/div[1]/a/div/cite')
            for elm in element:
                print(school_name)
                url_name = elm.text
                print(url_name)
            url_list.append(url_name)
            urls_df = urls.append(pd.DataFrame({'School Name': [school_name], 'URL': [url_name]}), ignore_index=True, sort = False)
            g_driver.get("http://amazon.com")
            time.sleep(3)
            g_driver.get("http://facebook.com")    
    urls.to_csv(create_path('University and College Websites update.csv'))
    return url_list

In [11]:
url_list = get_school_urls(urls, new_data['School'])
new_data.insert(4, "School URL", url_list) 
url_list = short_url(new_data['School URL'])
new_data.insert(5, "School URL short", url_list)
print('First half complete')
url_list = get_school_urls(urls, new_data['New School'])
new_data.insert(7, "New School URL", url_list) 
url_list = short_url(new_data['New School URL'])
new_data.insert(8, "New School URL short", url_list) 
new_data.head()

Boston College
http://bc.edu
Suffolk University
http://suffolk.edu
University of California Irvine
http://uci.edu
University of Detroit Mercy School of Law
www.law.udmercy.edu/
New York University
http://nyu.edu
University of Colorado Boulder
http://colorado.edu
University of California Los Angeles
http://ucla.edu
University of California Irvine
http://uci.edu
University of California Davis
http://ucdavis.edu
Whittier Law School
https://www.law.whittier.edu/
University of Minnesota
http://umn.edu
Suffolk University
http://suffolk.edu
University of California Irvine
http://uci.edu
University of Washington Seattle
https://www.washington.edu/
West Virginia University
http://wvu.edu
University of San Diego
http://sandiego.edu
University of San Francisco
http://usfca.edu
Texas A&M University
http://tamu.edu
University of Virginia
http://virginia.edu
DePaul University
http://depaul.edu
Louisiana State University
http://lsu.edu
University of Missouri Kansas City
http://umkc.edu
Columbia Unive

Boston University
http://bu.edu
University of the Pacific McGeorge School of Law
https://www.mcgeorge.edu/
University of Houston
www.uh.edu/
University of California Berkeley
https://www.berkeley.edu/
University of Connecticut
http://uconn.edu
University of Chicago
http://uchicago.edu
Columbia University 
https://www.columbia.edu/
Southern Methodist University
http://smu.edu
University of North Carolina Chapel Hill
https://www.unc.edu/
Rutgers University Camden
https://www.camden.rutgers.edu/
New York University
http://nyu.edu
University of Maryland
http://umaryland.edu
University of Iowa
https://uiowa.edu/
University of Akron
https://www.uakron.edu/
University of Missouri
https://missouri.edu/
University of Minnesota Twin Cities
https://twin-cities.umn.edu/
University of California Berkeley
https://www.berkeley.edu/
Washington & Lee University
https://www.wlu.edu/
University of Illinois
http://uillinois.edu
Seton Hall University
http://shu.edu
Northwestern University
http://northweste

University of Missouri Columbia
https://missouri.edu/
Northeastern University
http://northeastern.edu
University of Florida Gainesville
www.ufl.edu/
University of Pennsylvania
http://upenn.edu
University of Pennsylvania
http://upenn.edu
University of Akron
https://www.uakron.edu/
University of Pennsylvania
http://upenn.edu
University of Houston
www.uh.edu/
Southern Methodist University
http://smu.edu
University of Southern California
http://usc.edu
University of Richmond
http://richmond.edu
Northwestern University
http://northwestern.edu
University of Virginia
http://virginia.edu
Southern Methodist University
http://smu.edu
University of Notre Dame
http://nd.edu
University of Hawaii
https://wustl.edu/
Yale University
http://yale.edu
University of California Irvine
http://uci.edu
Yale University
http://yale.edu
Boston College
http://bc.edu
Harvard University
http://harvard.edu
University of Nevada Las Vegas
http://unlv.edu
University of California Berkeley
https://www.berkeley.edu/
Broo

Wayne State University
https://wayne.edu/
Harvard University
http://harvard.edu
Georgetown University
http://georgetown.edu
McGill University
https://www.mcgill.ca/
University of Notre Dame
http://nd.edu
University of California Hastings
https://www.uchastings.edu/
Emory University
http://emory.edu
Duke University
http://duke.edu
University of California Irvine
http://uci.edu
Loyola University Chicago
http://luc.edu
College of William & Mary
https://wustl.edu/
University of Nevada Las Vegas
http://unlv.edu
University of Virginia
http://virginia.edu
University of Colorado Boulder
http://colorado.edu
University of California Los Angeles
http://ucla.edu
Florida State University
http://fsu.edu
Columbia University
http://columbia.edu
University of Nevada Las Vegas
http://unlv.edu
Temple University
http://temple.edu
University of Houston
www.uh.edu/
University of Nevada Las Vegas
http://unlv.edu
University of New Hampshire
http://unh.edu
University of Illinois
http://uillinois.edu
University

,First Name,Last Name,Title,School,School URL,School URL short,New School,New School URL,New School URL short
0,Richard,Albert,NA,Boston College,http://bc.edu,bc,"University of Texas, Austin",https://www.utexas.edu/,utexas
1,Hilary,Allen,NA,Suffolk University,http://suffolk.edu,suffolk,American University,http://american.edu,american
2,Olufunmilayo,Arewa,NA,"University of California, Irvine",http://uci.edu,uci,Temple University,http://temple.edu,temple
3,Khaled A.,Beydoun,NA,University of Detroit Mercy School of Law,www.law.udmercy.edu/,law.udmercy,"University of Arkansas, Fayetteville",https://www.uark.edu/,uark
4,Joshua,Blank,NA,New York University,http://nyu.edu,nyu,"University of California, Irvine",http://uci.edu,uci


In [13]:
new_data.to_csv(create_path('multi_school_data', 'new_professor_name_data.csv'))
# if 'american' in 'https://www.wcl.american.edu/community/faculty/profile/hjallen/':
#     print('True')
new_data[new_data['First Name'] == 'Amy']

,First Name,Last Name,Title,School,School URL,School URL short,New School,New School URL,New School URL short
128,Amy,Schmitz,NA,"University of Colorado, Boulder",http://colorado.edu,colorado,"University of Missouri, Columbia",https://missouri.edu/,nyu
199,Amy,Landers,NA,"University of the Pacific, McGeorge School of Law",https://www.mcgeorge.edu/,mcgeorge,Drexel University,http://drexel.edu,drexel


In [1]:
os.getcwd()
stats = ['Cited by Cases','Cited by Articles','Accessed (Past 12 Months)','Cited by Articles (Past 10 Years)', 'Cited by Articles (Past 1-2 years)', 'ScholarCheck Rank', 'Average Citations per Article', 'Average Citations per Document', 'Self-Citations']
path = create_path('multi_school_data')
files = os.listdir(path)
school_name = 'multi_school'
delay = 5
current_papers = os.listdir(create_path('author_papers'))
current_stats = os.listdir(create_path('school_stats'))
current_skip = os.listdir(create_path('skipped_names'))
for file in files:
    print(file)
    main_df = pd.DataFrame()
    file1 = check_df(current_stats, school_name)
    if file1 != '':
        main_df = pd.read_csv(create_path('school_stats', file1))
    skip_df = pd.DataFrame(columns = ['Full Name', 'School'])
    file2 = check_df(current_skip, school_name)
    if file2 != '':
        skip_df = pd.read_csv(create_path('skipped_names', file2))
    data = pd.read_csv(create_path('multi_school_data', file))   
    for i in range(len(data)):
        #Get name and information from the database
        mid_first_name = data['First Name'][i]
        last_name = data['Last Name'][i]
        full_name = mid_first_name + ' ' +  last_name
        done = check_files(mid_first_name, last_name, current_papers)
        #old url
        school_url = [data['School URL short'][i], data['New School URL short'][i]]
        if done:
            print('File for ' + full_name + ' has already been created.')
            continue
        school = data['New School'][i]
        title = data['Title'][i]
        page_name = []
        err_fm_names = []
        df_sub = pd.DataFrame()
        #Search by author to find potential alternative first and middle names:
        fm_names, err_fm_names = search_names(mid_first_name, last_name, school_url)
        print(fm_names)
        if not fm_names:
            print('Name ' + full_name + ' was not found')
            skip_df = skip_df.append(pd.DataFrame([[full_name, school, title]], columns = ['Full Name', 'School', 'Title']))
            
        for fm_name in fm_names:
            link = 'https://heinonline-org.proxy01.its.virginia.edu/HOL/AuthorProfile?action=edit&search_name=' + last_name +  '%2C ' + fm_name + '&collection=journals'
            driver.get(link)
            soup=bs.BeautifulSoup(driver.page_source, 'lxml')
            table_rows = soup.findAll('td', {'style': 'text-align:right;'})
            full_name = fm_name + ' ' +  last_name
            webpage_wait('//*[@id="page_content"]/div[1]/div/div[1]/div[1]')
            fm_names, err_fm_names = similar_names(fm_names, err_fm_names, fm_name, last_name)
            cur_page = driver.find_element_by_xpath('//*[@id="page_content"]/div[1]/div/div[1]/div[1]').text
            if not table_rows:
                got_page = False
                new_names = False
                link_index = 1
                while new_names == False:
                    try:
                        if link_index == 1:
                            element =driver.find_element_by_xpath('//*[@id="page_content"]/div[2]/div/ul/li/a')
                        else: 
                            element =driver.find_element_by_xpath('//*[@id="page_content"]/div[2]/div/ul/li[' + str(link_index) + ']/a')            
                        new_fm_name = element.text.split(', ')[1]
                        new_last_name = element.text.split(', ')[0]
                        if last_name == new_last_name and mid_first_name in new_fm_name:
                            if not new_fm_name in fm_names:
                                check_google(new_fm_name, last_name, school_url)
                                fm_names.append(new_fm_name)
                            
                    except: 
                        new_names = True
                        got_page = True
                        if not scraped_papers:
                            print('Name ' + full_name + ' is not in the database. You may be missing a middle initial.')
                            skip_df = skip_df.append(pd.DataFrame([[full_name, school, title]], columns = ['Full Name', 'School', 'Title']))
                        else:
                            print('No remaining pages to scrape from {}.'.format(full_name))
                    link_index += 1
                    
            elif table_rows and cur_page not in page_name: 
                element = driver.find_element_by_xpath('//*[@id="page_content"]/div[1]/div/div[2]')
                table_element = element.text.split('\n')
                number_list = []
                rank_list = []
                stat_list = []
                for stat in stats:
                    find_index = [table_element.index(s) for s in table_element if stat == s]
                    if find_index:
                        my_list = table_element[find_index[0]+1].split(' ')
                        number_list.append(my_list[0])
                        stat_list.append(stat)
                        if len(my_list) > 1:
                            rank_list.append(my_list[-1])
                    if stat == 'Self-Citations':
                        find_index = [table_element.index(s) for s in table_element if stat in s]
                        if find_index:
                            stat_list.append(stat)
                            number_list.append(table_element[find_index[0]].split(' ')[1])
                zip_number_list = list(zip(stat_list, number_list))
                zip_rank_list = list(zip(stat_list, rank_list))
                number_dict = create_dataframe_dict(zip_number_list)
                rank_dict = create_dataframe_dict(zip_rank_list)
                df_number = pd.DataFrame.from_dict(number_dict, orient='index').transpose()
                df_rank = pd.DataFrame.from_dict(rank_dict, orient='index').transpose()
                df_number = df_number.replace('na', '0')
                df_number = df_number.replace('', '0')
                df_number = df_number.replace(' ', '0')
                df_number = remove_commas(df_number)
                df_number = df_number.astype(float)
                if df_sub.empty:
                    df_sub = df_number
                else: 
                    df_sub = df_sub.add(df_number)
                
                title_index = 3
                stats_index = 4
                topic_index = 0
                topic_div_index = 0
                topic_array = soup.findAll('div', {'class': 'topics'})
                element = title_index
                df = pd.DataFrame(columns = ['Title', 'Author(s)', 'Journal', 'Cited (articles)', 'Cited (cases)', 'Accessed', 'Topics'])
                while element:
                    data_stream = []
                    x_path_title = '//*[@id="save_results"]/div/div/div/div[' + str(title_index) + ']/div[2]/dt[1]/div'
                    element = driver.find_elements_by_xpath(x_path_title)
                    #Get title:
                    if element:
                        scraped_papers = True
                        for elm in element:
                            data_stream.append(elm.text)
                        element = driver.find_elements_by_xpath('//*[@id="save_results"]/div/div/div/div[' + str(title_index) + ']/div[2]')
                        for elm in element:
                            my_list = elm.text
                        if [a for a in my_list.split('\n') if last_name in a]:   
                            data_stream.append([a for a in my_list.split('\n') if last_name in a][0])
                        else:
                            data_stream.append('na')
                        if [a for a in my_list.split('\n') if 'Vol.' in a]:
                            data_stream.append([a for a in my_list.split('\n') if 'Vol.' in a][0])
                        else:
                            data_stream.append('na')
                        element = driver.find_elements_by_xpath('//*[@id="save_results"]/div/div/div/div[' + str(stats_index) + ']/div[2]/div')
                        for elm in element:
                            cited_text = elm.text
                        article_citations = 'na'
                        case_citations = 'na'
                        accessed = 'na'
                        if not isinstance(cited_text, list):
                            cited_text = cited_text.split('\n')
                            cited_text
                            for stat in cited_text:
                                if 'Article' in stat:
                                    article_citations = int(re.search(r'\d+', stat).group())
                                if 'Case' in stat:
                                    case_citations = int(re.search(r'\d+', stat).group())
                                if 'Accessed' in stat:
                                    accessed = int(re.search(r'\d+', stat).group())
                        data_stream.append(article_citations)
                        data_stream.append(case_citations)
                        data_stream.append(accessed)
                        if 'Topics:' in my_list:
                            data_stream.append(topic_array[topic_div_index].text.split(':')[1])
                            topic_div_index +=1
                        else:
                            data_stream.append('na')
                        df = df.append(pd.DataFrame([data_stream], columns = ['Title', 'Author(s)', 'Journal', 'Cited (articles)', 'Cited (cases)', 'Accessed', 'Topics']))
                        stats_index +=4
                        title_index += 4
                        page_name.append(cur_page)
                        #Check that next paper exists:
                        x_path_title = '//*[@id="save_results"]/div/div/div/div[' + str(title_index) + ']/div[2]/dt[1]/div'
                        element = driver.find_elements_by_xpath(x_path_title)
                df.to_csv(create_path('author_papers', '{}_{}_papers.csv'.format(full_name, school)))
                time.sleep(3)
            print('No remaining pages to scrape for {}.'.format(fm_name + ' ' + last_name))  
        if not df_sub.empty:
            my_dict = {'Person': [full_name], 'School': [school], 'Type': ['number']}
            name_data = pd.DataFrame(my_dict)
            df_sub = pd.concat([name_data, df_sub], sort = False, axis = 1)
            main_df = pd.concat([main_df, df_sub], sort = False)
            main_df.replace(0, 'na')
            main_df.to_csv(create_path('school_stats', '{}_stats.csv'.format(school_name)))                
        skip_df.to_csv(create_path('skipped_names', '{}_skipped.csv'.format(school_name)))
                    
skip_df.to_csv(create_path('skipped_names', '{}_skipped.csv'.format(school_name)))
main_df.replace(0, 'na')
main_df.to_csv(create_path('school_stats', '{}_stats.csv'.format(school_name)))

NameError: name 'os' is not defined

#### file

In [92]:
files[0]

'missed_names.csv'

In [1]:
last_name = 'Cohen'
title_index = 3
stats_index = 4
topic_index = 0
topic_div_index = 0
topic_array = soup.findAll('div', {'class': 'topics'})
element = title_index
df = pd.DataFrame(columns = ['Title', 'Author(s)', 'Journal', 'Cited (articles)', 'Cited (cases)', 'Accessed', 'Topics'])
while element:
    data_stream = []
    x_path_title = '//*[@id="save_results"]/div/div/div/div[' + str(title_index) + ']/div[2]/dt[1]/div'
    element = driver.find_elements_by_xpath(x_path_title)
    #Get title:
    if element:
        scraped_papers = True
        for elm in element:
            data_stream.append(elm.text)
        element = driver.find_elements_by_xpath('//*[@id="save_results"]/div/div/div/div[' + str(title_index) + ']/div[2]')
        for elm in element:
            my_list = elm.text
        if [a for a in my_list.split('\n') if last_name in a]:   
            data_stream.append([a for a in my_list.split('\n') if last_name in a][0])
        else:
            data_stream.append('na')
        if [a for a in my_list.split('\n') if 'Vol.' in a]:
            data_stream.append([a for a in my_list.split('\n') if 'Vol.' in a][0])
        else:
            data_stream.append('na')
        element = driver.find_elements_by_xpath('//*[@id="save_results"]/div/div/div/div[' + str(stats_index) + ']/div[2]/div')
        for elm in element:
            cited_text = elm.text
        article_citations = 'na'
        case_citations = 'na'
        accessed = 'na'
        if not isinstance(cited_text, list):
            cited_text = cited_text.split('\n')
            cited_text
            for stat in cited_text:
                if 'Article' in stat:
                    article_citations = int(re.search(r'\d+', stat).group())
                if 'Case' in stat:
                    case_citations = int(re.search(r'\d+', stat).group())
                if 'Accessed' in stat:
                    accessed = int(re.search(r'\d+', stat).group())
        data_stream.append(article_citations)
        data_stream.append(case_citations)
        data_stream.append(accessed)
        if 'Topics:' in my_list:
            data_stream.append(topic_array[topic_div_index].text.split(':')[1])
            topic_div_index +=1
        else:
            data_stream.append('na')
        df = df.append(pd.DataFrame([data_stream], columns = ['Title', 'Author(s)', 'Journal', 'Cited (articles)', 'Cited (cases)', 'Accessed', 'Topics']))
        stats_index +=4
        title_index += 4
        page_name.append(cur_page)
        #Check that next paper exists:
        x_path_title = '//*[@id="save_results"]/div/div/div/div[' + str(title_index) + ']/div[2]/dt[1]/div'
        element = driver.find_elements_by_xpath(x_path_title)

NameError: name 'soup' is not defined

In [ ]:
# df_sub = pd.DataFrame()
# main_df = pd.DataFrame()
stats = ['Cited by Cases','Cited by Articles','Accessed (Past 12 Months)','Cited by Articles (Past 10 Years)', 'Cited by Articles (Past 1-2 years)', 'ScholarCheck Rank', 'Average Citations per Article', 'Average Citations per Document', 'Self-Citations']
last_name = 'Dauber'
mid_first_name = 'Michele Landis'
full_name = mid_first_name + ' ' + last_name
school = 'Arizona'
link = 'https://heinonline-org.proxy01.its.virginia.edu/HOL/AuthorProfile?action=edit&search_name=' + last_name +  '%2C ' + mid_first_name + '&collection=journals'
driver.get(link)
element = driver.find_element_by_xpath('//*[@id="page_content"]/div[1]/div/div[2]/div[1]')
table_element = element.text.split('\n')
number_list = []
rank_list = []
stat_list = []
for stat in stats:
#     print(stat)
    find_index = [table_element.index(s) for s in table_element if stat == s]
    if find_index:
        my_list = table_element[find_index[0]+1].split(' ')
        number_list.append(my_list[0])
        stat_list.append(stat)
        if len(my_list) > 1:
            rank_list.append(my_list[-1])
    if stat == 'Self-Citations':
        find_index = [table_element.index(s) for s in table_element if stat in s]
        if find_index:
            stat_list.append(stat)
            number_list.append(table_element[find_index[0]].split(' ')[1])
zip_number_list = list(zip(stat_list, number_list))
zip_rank_list = list(zip(stat_list, rank_list))
number_dict = create_dataframe_dict(zip_number_list)
rank_dict = create_dataframe_dict(zip_rank_list)
df_number = pd.DataFrame.from_dict(number_dict, orient='index').transpose()
df_rank = pd.DataFrame.from_dict(rank_dict, orient='index').transpose()
df_number = df_number.replace('na', '0')
df_number = df_number.replace('', '0')
df_number = df_number.replace(' ', '0')
df_number = df_number.astype(float)
if df_sub.empty:
    df_sub = df_number
else: 
    df_sub = df_sub.add(df_number)
df_sub
my_dict = {'Person': [full_name], 'School': [school], 'Type': ['number']}
name_data = pd.DataFrame(my_dict)
df_sub = pd.concat([name_data, df_sub], sort = False, axis = 1)
main_df = pd.concat([main_df, df_sub])

In [374]:
    main_df

,Person,School,Type,Cited by Cases,Cited by Articles,Accessed (Past 12 Months),Cited by Articles (Past 10 Years),Cited by Articles (Past 1-2 years),ScholarCheck Rank,Average Citations per Article,Average Citations per Document,Self-Citations
0,Michele Landis Dauber,Arizona,number,0.0,97.0,38.0,45.0,2.0,0.0,97.0,24.25,0.0


In [ ]:
df_number = df_number.replace('na', '0')
df_number = df_number.replace(' ', '0')
df_number = df_number.astype(float)
df_number.add(df_number)

In [319]:
my_dict = {'Person': ['adsfdsa'], 'School': ['dsafdsa'], 'Type': ['sdafdsaf']}
name_data = pd.DataFrame(my_dict)
pd.concat([name_data, df_number], sort = False, axis = 1)

,Person,School,Type,Cited by Cases,Cited by Articles,Accessed (Past 12 Months),Cited by Articles (Past 10 Years),Cited by Articles (Past 1-2 years),ScholarCheck Rank,Average Citations per Article,Average Citations per Document,Self-Citations
0,adsfdsa,dsafdsa,sdafdsaf,3.0,493.0,645.0,296.0,36.0,0.0,22.41,15.9,80.0


In [57]:
element = driver.find_element_by_xpath('//*[@id="page_content"]/div[1]/div/div[2]/div[1]')
stats = ['Cited by Cases','Cited by Articles','Accessed (Past 12 Months)','Cited by Articles (Past 10 Years)', 'Cited by Articles (Past 1-2 years)', 'ScholarCheck Rank', 'Average Citations per Article', 'Average Citations per Document', 'Self-Citations']
table_element = element.text.split('\n')
number_list = []
rank_list = []
stat_list = []
for stat in stats:
#     print(stat)
    find_index = [table_element.index(s) for s in table_element if stat == s]
    if find_index:
        my_list = table_element[find_index[0]+1].split(' ')
        number_list.append(my_list[0])
        stat_list.append(stat)
        if len(my_list) > 1:
            rank_list.append(my_list[-1])
    if stat == 'Self-Citations':
        find_index = [table_element.index(s) for s in table_element if stat in s]
        stat_list.append(stat)
        number_list.append(table_element[find_index[0]].split(' ')[1])

print(stat_list)
print(number_list)
print(rank_list)

['Cited by Articles', 'Accessed (Past 12 Months)', 'Cited by Articles (Past 10 Years)', 'Cited by Articles (Past 1-2 years)', 'ScholarCheck Rank', 'Average Citations per Article', 'Average Citations per Document', 'Self-Citations']
['99', '212', '99', '18', '', '9.90', '9.00', '6']
['10,679', '8,754', '4,126', '1,898', '7,876']


In [364]:
df1 = pd.DataFrame({'a': ['1342'], 'b':['na'], 'c':['2']})
df1 = df1.replace('na', '0')
df1 = df1.astype(float)
df2 = pd.DataFrame({'a': ['1'], 'b': ['2'], 'd':['na']})
df2 = df2.replace('na', '0')
df2 = df2.astype(float)
df_out = df1.add(df2, fill_value=0)
# # df_out.replace(0, 'na')

In [365]:
df_out

,a,b,c,d
0,1343.0,2.0,2.0,0.0


In [357]:
df1

,a,b,c
0,1342,0,2


In [341]:
'1,342'.replace(',', '')

'1342'